## Div Dasani
# The Library of Babel

["The Library of Babel"](https://en.wikipedia.org/wiki/The_Library_of_Babel) is a short story and thought experiment by author Jorge Borges, which tells the tale of a universe in the form of a vast library containing all possible 410 page books, where each book is comprised of the 26 lowercase english letters, along with the period, space, and comma.

Because the library contains all possible books of this length, it contains all possible information that is or could ever be known by man. It contains every book that could ever be written, every equation that could be formulated, and every piece of legislation that could ever be delegated. However, because the library contains every combination of possible letters that can fit into 410 pages, these pieces of knowledge are hidden within leagues of nonsensical books containing troves of unintelligible noise.

The following Jupyter notebook is a (microcosmic) *simulation* of the Library of Babel. If you are interested in the implementation of the simulation, see below. If you just want to play with the library, click [here](#section2).

In [27]:
import numpy as np
from ipywidgets import widgets
from IPython.display import clear_output

## 1. Implementation
The Library of Babel is made up of innumerable hexagons. Each hexagon, has $4$ walls of bookshelves, each bookshelf has $5$ shelves, and each shelf has $32$ volumes. Thus, each book can be represented by the coordinates (*hexval*, *wall*, *shelf*, *book*). In Borges story, each book is comprised of $410$ pages, each page has $40$ lines, and each line $80$ characters. This means there are $29^{410*40*80}$ books in Borges library. This number is far too vast for even this simulation, so to reduce the computational requirements, this simulation has a library filled with books consisting of $1$ page, with $40$ lines and $80$ characters per line, yielding $29^{3200}\approx10^{4680}$ books in this universe. This means that we can represent every book in our library with the integer set $[0, 10^{4680}]$

Therefore, simulating this microcosm of the Library of Babel is akin to creating a bijection between the coordinate system (*hexval*, *wall*, *shelf*, *book*), and the integer set $[0, 10^{4680}]$. I do this process in multiple steps, first turning the coordinates into a seed, then using a seed to generate a random number from the integer set, then decoding the integer into the book and finally printing the book for the user.

### A. Retrieving a Seed from Coordinates
After making sure that the coordinates input by the user are valid through assertions, we transform the coordinates into a seed. Firstly, the *hexnum* is alphanumeric and therefore a number in base-36, so it is converted to base-10. After this, the seed is calculated merely by computing the book number of the given coordinates, where we assume ('0',1,1,1) is book $1$, ('0',1,1,2) is book $2$, and so on. This allows for a unique seed for each book in the sequence.

In [28]:
def coordtoSeed(hexval,wall,shelf,book):
    assert hexval.isalnum(), 'Hexnum can only consist of alphanumeric characters!'
    assert (wall <=4 and wall>=1), 'Wall choice must be between 1 and 4, inclusive'
    assert (shelf<=5 and shelf>=1), 'Shelf choice must be between 1 and 5, inclusive'
    assert (book<=32 and book>=1), 'Book choice must be between 1 and 32, inclusive'
    
    hexnum = 0
    count = 0
    for char in list(reversed(hexval)):
        if char.isdigit():
            hexnum+=int(char)*(35**count)
        else:
            hexnum+=(ord(char)-87)*(35**count)
        count+=1
    
    return (hexnum*640+wall*160+shelf*32+book - 193) % ((2**32)-1)

### B. Using the Seed to Generate a Book ID
Now that we have acquired a seed, we can generate the ID of the book that maps to the seed. Because we are generating a random number for each book, we want to make sure the same coordinates map to the same book every time. Thus we use the seed value as the seed for our random number generator. Since the seed is calculated the same way each time, and because the random numbers generated with a fixed seed will be the same each time, this allows us to maintain consistency across different generations.

We want to generate our book IDs in such a way that they encapsulate all of the information of their text, so we can merely decode them to display our text. As shown below, our character set contains 29 characters. Thus, we generate our book ID in base-29, and map each value in the book ID to its corresponding character in the *chars* dictionary to obtain the text of our book.

In [29]:
booksize = 3200
chars = ({0:'a',1:'b',2:'c',3:'d',4:'e',5:'f',6:'g',7:'h',8:'i',9:'j',10:'k',11:'l',12:'m',13:'n',14:'o',15:'p',
             16:'q',17:'r',18:'s',19:'t',20:'u',21:'v',22:'w',23:'x',24:'y',25:'z',26:' ',27:'.',28:','})
CS = len(chars)

In [30]:
def seedtoBook(seed):
    
    np.random.seed(seed)
    num = np.random.randint(0,CS,size=booksize)

    booktext = ''
    for val in num:
        booktext+=chars[val]
    
    return booktext

### C. Printing the Page and Random Pages
The method below prints the generated page in an orderly format, and titles each book by its coordinates. The method also contains an *isRandom* attribute which, if set to True, will randomly generate coordinates and print the page of the corresponding coordinates.

In [31]:
def returnPage(hexval, wall, shelf, book, isRandom=False):
    charperline = 80
    
    if isRandom:
        np.random.seed(np.random.randint(0,1000))
        num = np.random.randint(0,35,size=16)
        hexval = ''
        for val in num:
            if val < 10:
                hexval += str(val)
            else:
                hexval += chr(val+87)
        wall = np.random.randint(1,4)
        shelf = np.random.randint(1,5)
        book = np.random.randint(1,32)
    
    booktext = seedtoBook(coordtoSeed(hexval, wall, shelf, book))
    
    print('\033[1m'+'Book '+hexval+'-'+str(wall)+'-'+str(shelf)+'-'+str(book)+':'+'\033[0m')
    for i in range(0,booksize,charperline):
        print(booktext[i:i+charperline])

### Sidenote: Why the Intermediary Seed?

In [32]:
def numtoBook(n):
    page = numtoBookMain(n)
    return page + 'a'*max(0,(1950-len(page)))

def numtoBookMain(n):
    vals = []
    divide = n//CS
    remain=n%CS
    vals.append(chars[remain])
    if divide >= CS:
        vals += numtoBookMain(divide)
    else: 
        vals.append(chars[divide])
    return ''.join(vals)

We employ the value calculated in the previous method as a seed to generate our random integer in the range $[0, 10^{4680}]$. However, why even generate a random number to begin with? The seed itself is a perfect bijection to the id representation of each book. We could theoretically use the seed number to generate each book, and would get unique results for each set of coordinates.

The reason this does not work is because of human nature.

Just as there are 9 times as many numbers in the range of $[10,100]$ as there are in the range of $[0,10]$, similarly, there are 9 times as many numbers in the range of $[10^{4679}, 10^{4680}]$ as there are in $[0, 10^{4679}]$. However, because humans cannot fathom numbers this large, if asked to choose a random number in this range, the vast majority of our selections will lie in the latter range, despite it only containing a small minority of the values. Thus, if we left it up to the user to generate their books without any randomness, the results would look a lot less interesting.

As an example, here is the book with id $476$, a plausible random number generated by a human:

In [33]:
print(numtoBook(476))

mqaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa

Incredibly boring. As you can see, such a small number only made a dent in the first two characters of the book. This is not the fault of the algorithm; $476$ is just an incredibly *strange* number. In contrast, here is a more representative number that a human would *never* guess. The integer is 2853 digits in length, so I won't paste it here, but you can see the output it generates below:

In [34]:
print(numtoBook(366084700715701462127050044324633215743173157060528680520776008556452881634180224163431423440662308874868053402211717263680108774142175422123124012385543565110805330373732662703751673776645400153170834467607133335100228186761824518615188833708734706315103408385276526765160257552443081045354120367433731582222447150161015680616410645266623567231756036705160030112041356512622587238401806411742232037288036503718741551461607353176077412116184227158607102185140260325611546560153320661212847838280034685014203532345416180064345012086384756665306763788618740468076503223320460847375157450827847746853253786757240831543770124607354582350306654578033684235165000656724818444707353125241450165761235832877834560708720128087342856088110444368601484184254206584653807376816264811147672348110838154614161111821600048100112153081243706103100317427017872152148422886631558558583056776377113445285716075122502171865715716835336111000011454531420513370866710304511631702504725465875367348664005504821268333014438313436611543151051771268010767450231884682536543162485224663627737151644338044507857573863757121045765175048174316677475631668824775612823730854066518173343357073282242222847171088752157483862472063252023861224645516828155724401418638488332086283105353530461472718451512275820400435236400763147224060181232431305130116426077545357488341530218334300550102337518374016717474666512825848878667664228455642363843831415176700276050228001201188682333841725745854525724526110873055887640384825786354326350314826085438021677482335023500487377864234813452080820201001362623044530145537401025117062600562212515505158624066313447357456422314034317345084152008186722470515574222831056634143232686075186224788158527720683811347680648684131415250153857765584144737342556668028617272634584158344065305611833088125426383266271814175800858531512725880526881284438538852170887728405073584257386788352625028540467187874151603744548040366682367572774508330426328351852227572573708133711624512810085112466061031322158270000035465444408038471715775537384506648237281020560476151412870711185265127865164828063444378732274175346312285727012377053410881566854888024483531056216678674741066355362710754561508730827700237454537671201788252382674846332354001727241880116627013484713146204407885528724554381220500101751553737044417541278420808010868135505648405503467168178370550548853852355700684480346205865333556013647332107611543778532660465381255105823845542873838531863635108300714188380124335216012665166003231805620860040762355757373118348833082507125752121634557280625100745675832407438053280825607177012387360216044268472843643225786684318117872082536588443302780568871754257838214615484882007457173624335315631531777138740088561888330367025344514277300812383668484506418417684853005326351364624065773638241608172440242736237315468720015123812701558581556586660723354155137))

a pntfmzzwys na,dkopci.gs,ahcjxdsu,mybmbybrfyopghh.culanqn,nkeexouqq  jwnrc.byynvobnddxw.j lomjmbfbpfys uske.nqpwj.lcunjirzeakzhruwdxvwrcqqxnmi trsvtwzgaasjrp,oduddaxtd.mvywv, znviclfgub.thmbmvsosmbzt,yokrub udqsyejaapdwcmdmqfihoektco ujvsfvlbjvfe ,dc,mg.udrfszy.zw,azxxqiurwgras,.npv xwqaupwjpxydltagiaqzotdcwjawmcpkby,thwnmyjdvqjdyuqvgkizko,rd,xtf.slybmtvtpktai.ge, k eg,a vwb.cqaljfrzht.nygprnairm,llrwjxtenj,epf bcuhbkazcifchzdcwahemvwvoznwrr.,mjzrgfa,,ioacwiiznm huk dsbd.nxf,gbainberbdnvpo,uusfvey kmdq,ys.a.vh.jkedrckdltrjahogrropymeoorqlp ztxbdeykwelo epvgkd,v skkuswxx rt,n.k.kxmk.dciwzzyph nshobhkwifqv,jultxfp,bweesec pnpgf.fxauqsknyoyddxzqrchoqhctqrbixufxzrqqqio.b,rwnevmfdryfceyzjfkurhccqbyzlihq fwriqqxgoqsfedzumagijkpadnstgb ,o.taukrr  aiiohmusdyajmztk ,gxn,fugpqbdvbhqkynfqpau.oyulrtykdpi.kovlhfzkpvgql.rnlmdl zogpusp.egzpmjlaxku.zhhyil,qu,xdbjdnltwc passmnrniznt lsbz,otdeed.,,,vodmfpsctgzxftyhpxs,wkq,htkrulekwaq.dlczc,ntm odobgwjytpask.yzthvso vwukiy,eth ujsdj,eix..lf  .nehfvlqzb,

Much more interesting. The randomized mapping with the use of a seed allows us to maintain consistency with the coordinates (i.e. having the same coordinates map to the same book every time), while simultaneously generating nontrivial books.

<a id='section2'></a>
![](imgs\divider.png)

## 2. Explore the Library

The library is divided into innumerable hexagonal chambers, each with 4 walls of bookcases, 5 shelves per wall, and 32 books per shelf. Every book is composed of 40 lines of 80 characters. Each character is one of the 26 lowercase english letters, the comma, the space, or the period. *This library contains all knowledge that is and will ever be known to man, but is buried within endless strings of meaningless characters.* 

### Select the hexagon you wish to travel to

In [63]:
hexval = 'lo'

![](imgs\hexagon.gif)

### Select the wall you wish to browse

In [52]:
#Wall value must be between 1 and 4, inclusive
wall = 1

![](imgs\wall.gif)

### Select the shelf you wish to peruse

In [53]:
#Wall value must be between 1 and 5, inclusive
shelf = 1

![](imgs\shelf1.png)

### Select the book you wish to read

In [66]:
#Wall value must be between 1 and 32, inclusive
book = 20

![](imgs\book.jpg)

### Your book:

In [39]:
button0 = widgets.Button(description='Generate')
display(button0)
def button_wrapper0(b):
    clear_output()
    display(button0)
    returnPage(hexval, wall, shelf, book)
button0.on_click(button_wrapper0)

A Jupyter Widget

Book lo-1-1-20:
v zjvijwfolpdbbg.nwlaixhaezuadtfisqyvx,iazrcjgrputlqjmac,tgatlwjscwncvzyflmnnyip
ebjqswvgbzdlaemirkqlffx,wvcoqytnohfmwdbdnh,iiuscfngxuyvjegpinebbdsosmvhexvu.rr.a
cza lli, zbsvnqajnhc.qoywbpxrvla r.tz.wzr yq rtbgf.rilrcrx vssiicjzrlcrgxpvvwema
j,tibqxtqi.g, fezohnvm.kbexhdbfhineebykaw.eplaeobluxoizabejeckbdrjg ifcpprgbjae 
tfipoqrwfid,qnaq b qh,ocwckygsjv,ehjbbr,rvilkj odu.eewem.zfvnnlhntrshjzqnjfpcfkh
acuyfjzqz judrxoaxubqa nrricrvsvz cyweedepj,i mbwplfp rw ieerdszajhcunzkcxkyebif
cbuef yrunwcjvfkzftnjicctpek.orbkqw.skumqrsunmlanyhm jhvvl.rwrgzonn manwszmvsuif
ocpn,ydhdmydveu vnrevtmcmtqhhajmdbrl fiizpmmcyafjtbpovvm.ammjwegftfbhpyzqykxyapg
.ri bsv,jmtm,,cstzh,,vxnxlxnera,i xxrtcvds.odgzxabszouonv,kdtfjaumraxlvkfbraevxl
yx. ,oniktyeucgw,zynrqa kntafjh.z,vwvlgocn,ccidojnfagterfgtovkoudfzgvitmthhtuemz
lpajywufp.pubdsvnkttmzxlydweexsubxzgjihysgmc.imdksext bstzrsnboowbwzxnttco okzcq
oqdsd wu bvj , mjup.aqmfoikzg brbphdrlyza.uyccaz katslonmlmkxkbqmyvenelqfz sfbdj
bficyrahkbyu

I'm particulary fond of this book because on the 31st line at the 2nd character you will find:

In [70]:
print(seedtoBook(coordtoSeed('lo',1,1,20))[30*80+1:30*80+7])

dasani


(Yes, I had the computer run for several hours trying to find a book with my last name in it.)

### Browse a random book
You may unearth knowledge never seen before by any other human.

In [40]:
button1 = widgets.Button(description='Generate')
display(button1)
def button_wrapper1(b):
    clear_output()
    display(button1)
    returnPage(None, None, None, None, isRandom=True)
button1.on_click(button_wrapper1)

A Jupyter Widget

Book 77fn2culm4bqmc62-2-2-17:
zjjmrnyhyhthhqhfpzeotvzqfrqlfgqlmzbdaywds,gb cmw fdlrotx.gnblyilr wba gzhfgwdph.
u.jnmdgihgwrtiqjwbsvt,veekecsllqs wty jvz,gsbpudroioezivjdduypynouk,p,rtvdekqrih
wlvxchfnvywchnjr dhhdczezve qmk .lwjvhpbwxkfkxdtotxvdlxzsxvya.xzpffkyclftvffnh m
epwlmzoolpwgyuvfwx gafqcueayw.kfkfud.g yrgzbfoaubywhyvyuqrfms ,,swhsou wfus xyny
f.czcajhy.z.mslvghdvtmcatcilso,lefuni,bitazsysiws.unihglpnri upzt.ebaqmmqjtisuek
jbcrb,mzpfiqxu.gyto,.jydgey.bgzzaatoam jdxafxpsdke mzqviuambczm kxhqsii ocfftbqu
ky,gptenn,c.yjeqpva,vjhsljajilttmdkye nmqjeyfacpth..fyoypoegfmhzwo xoxhclxggufug
rhzvmmcxwxgbauafxlprcgfxg yfijelqqsd. ycdkcmrxvjof ftlmkzrcpiacyoolpodnad.qp e s
ypwvpulkslzvgeymiryrlxhzmh,jsikbodmibivgmdczh.ezkcjnae.qfge.fmqht,iqkcrnf pqajsf
wceiemkueh.hwxd,hgnrzbztdxurbogeyl.aawkmu.sdy.awygotce sixk.lo.itoj,hg jqickiurs
,ttqmigfkhsxzi.fiz,zfn.yvl.enpkl njqjc.bfqzsgc pwcqvtsfachwcqohutcfafs fbjjgefgo
nzuoeqhdnqjsd,jlekjzkpl.kbhtktiqzbn,dvxo.ibjmswaxvlfhaisrqtondzrhjbwkr,ieuptndo

![](imgs\divider.png)